# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [25]:
!pip install xgboost==0.7.post4

In [52]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet') # download for lemmatization
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import pickle

import warnings
warnings.simplefilter("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
# load data from database
engine = create_engine('sqlite:////home/workspace/ETL.db')
df = pd.read_sql_table('ETL', engine)
# Set the message column as X
X = df.message
# Set the other 36 categories as Y
Y = df[df.columns[4:]]

In [28]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [29]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [30]:
def tokenize(text):
    ''' This function normalize and tokenize the text'''
    
    # Normalize the text (case normalization & punctuation removal)
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize the text by split text into words using NLTK
    words = word_tokenize(text)
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Stemming by using the `PorterStemmer()` to perform stemming on the words processed above.
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    # Lemmatization using the 'WorldNetLemmatizer()' to perform lemmatization on the `words` variable.
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]

    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [31]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    # Since we want to classify 36 category, the 'MultiOutputClassifier' is used with the 'RandomForestClassifier', 
    #   which does not support multi-target classification. 
    # Reference: https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [32]:
# Perform train text split
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# Fit and transform the training data using the pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
# Predict on test data
predicted = pipeline.predict(X_test)

# Report the f1 score, precision and recall for each output category of the dataset using 'classification_report'
# References: 
# - https://muthu.co/understanding-the-classification-report-in-sklearn/
# - https://sparkbyexamples.com/pyspark/pyspark-get-number-of-rows-or-columns/
for i in range(len(y_test.columns)):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], predicted[:,i]))

related :
             precision    recall  f1-score   support

          0       0.65      0.48      0.55      1534
          1       0.85      0.92      0.88      4970
          2       0.27      0.34      0.30        50

avg / total       0.80      0.81      0.80      6554

request :
             precision    recall  f1-score   support

          0       0.90      0.98      0.93      5439
          1       0.79      0.45      0.57      1115

avg / total       0.88      0.89      0.87      6554

offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6532
          1       0.00      0.00      0.00        22

avg / total       0.99      1.00      0.99      6554

aid_related :
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3811
          1       0.76      0.60      0.67      2743

avg / total       0.75      0.75      0.75      6554

medical_help :
             precision    reca

### 6. Improve your model
Use grid search to find better parameters. 

In [34]:
# References:
# - https://towardsdatascience.com/random-forest-hyperparameters-and-how-to-fine-tune-them-17aee785ee0d
# - https://medium.com/@benfenison/gridsearching-a-random-forest-classifier-fc225609699c
parameters = {
    'clf__estimator__n_estimators': [5, 10], # The Nº of Decision Trees in the forest
    # Commented the below due to the running time...
#     'clf__estimator__n_estimators': [50, 100, 200], # The Nº of Decision Trees in the forest 
#     'clf__estimator__criterion': ['gini','entropy'] # The criteria with which to split on each node (Gini or Entropy for a classification task) 
}

# Perform grid search
# Added n_jobs=-1 to use multiple cores of the processor which will speed up the process. Reference: https://stackoverflow.com/questions/17455302/gridsearchcv-extremely-slow-on-small-dataset-in-scikit-learn
cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [43]:
def display_results(cv, y_test, y_pred):
    ''' This function displays the accuracy, precision, and recall of each class'''
    labels = np.unique(y_pred)
    for i in range(len(y_test.columns)):
        print(y_test.columns[i], ':')
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    accuracy = (y_pred == y_test).mean()
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

In [36]:
# Fit the tuned model
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [45]:
# Predict on test data
tuned_predicted = cv.predict(X_test)

In [46]:
# Display the results
display_results(cv, y_test, tuned_predicted)

related :
             precision    recall  f1-score   support

          0       0.63      0.45      0.53      1534
          1       0.84      0.92      0.88      4970
          2       0.47      0.14      0.22        50

avg / total       0.79      0.80      0.79      6554

request :
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5439
          1       0.80      0.43      0.56      1115

avg / total       0.88      0.89      0.87      6554

offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6532
          1       0.00      0.00      0.00        22

avg / total       0.99      1.00      0.99      6554

aid_related :
             precision    recall  f1-score   support

          0       0.74      0.85      0.79      3811
          1       0.74      0.59      0.66      2743

avg / total       0.74      0.74      0.74      6554

medical_help :
             precision    reca

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [54]:
# Change the machine learning algorithm to be XGBoost
# References: 
# - https://www.datacamp.com/tutorial/xgboost-in-python
# - https://stackoverflow.com/questions/40916939/xgboost-for-multilabel-classification

improved_pipeline = Pipeline([
    ("vect", CountVectorizer(tokenizer = tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(xgb.XGBClassifier(objective='binary:logistic')))
])

improved_parameters = {
    'clf__estimator__learning_rate': [0.0003,0.001,0.01], 
    'clf__estimator__n_estimators': [5, 10]
}

# Perform grid search
improved_cv = GridSearchCV(improved_pipeline, param_grid = improved_parameters, n_jobs = -1)

In [55]:
# Fit the improved tuned model
improved_cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...eg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__learning_rate': [0.0003, 0.001, 0.01], 'clf__estimator__n_estimators': [5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [56]:
# Predict on test data
improved_predicted = improved_cv.predict(X_test)

# Display the improved model results
display_results(improved_cv, y_test, improved_predicted)

related :
             precision    recall  f1-score   support

          0       0.63      0.08      0.14      1534
          1       0.77      0.99      0.86      4970
          2       0.00      0.00      0.00        50

avg / total       0.73      0.77      0.69      6554

request :
             precision    recall  f1-score   support

          0       0.87      0.98      0.93      5439
          1       0.79      0.31      0.44      1115

avg / total       0.86      0.87      0.84      6554

offer :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6532
          1       0.00      0.00      0.00        22

avg / total       0.99      1.00      0.99      6554

aid_related :
             precision    recall  f1-score   support

          0       0.68      0.87      0.76      3811
          1       0.70      0.42      0.53      2743

avg / total       0.69      0.68      0.66      6554

medical_help :
             precision    reca

### 9. Export your model as a pickle file

In [58]:
# I chose the tuned model since it performance is better than the others.
# Reference: https://practicaldatascience.co.uk/machine-learning/how-to-save-and-load-machine-learning-models-using-pickle
pickle.dump(cv, open('tuned_model.pkl', 'wb'))
# pickle.dump(improved_cv, open('improved_model.pkl', 'wb'))
# pickle.dump(pipeline, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.